# 繼承
* 繼承內健型態的陷阱
* 多重繼承與方法解析順序

In [3]:
class DoppelDict(dict):
    def __setitem(self, key, value):
        super().__setiem__(key,[value] *2)

In [4]:
dd = DoppelDict(one = 1)

In [5]:
dd

{'one': 1}

In [8]:
dd['two'] = 2 

In [7]:
dd

{'one': 1, 'two': 2}

In [9]:
dd.update(three=3)

In [10]:
dd

{'one': 1, 'two': 2, 'three': 3}

In [12]:
import collections
class DoppelDict2(collections.UserDict):
    def __setitem(self, key, value):
        super().__setitem__(key, [value] *2)

In [13]:
dd = DoppelDict2(one=1)
dd

{'one': 1}

In [14]:
dd['two']=2

In [15]:
dd

{'one': 1, 'two': 2}

## 多重繼承與方法順序


In [16]:
class A:
    def ping(self):
        print('ping:', self)


class B(A):
    def pong(self):
        print('pong:', self)


class C(A):
    def pong(self):
        print('PONG:', self)


class D(B, C):

    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


In [17]:
d =D()
d.pong()

pong: <__main__.D object at 0x7f9f6c18bc88>


In [19]:
C.pong(d)

PONG: <__main__.D object at 0x7f9f6c18bc88>


* Python 便歷繼承關係圖時候會遵循特定順序 MRO: method resolution order
* 每個類別都有一個屬性 __mro__ , 他會按照順序保存超類別

In [20]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [21]:
d.pingpong()

ping: <__main__.D object at 0x7f9f6c18bc88>
post-ping: <__main__.D object at 0x7f9f6c18bc88>
ping: <__main__.D object at 0x7f9f6c18bc88>
pong: <__main__.D object at 0x7f9f6c18bc88>
pong: <__main__.D object at 0x7f9f6c18bc88>
PONG: <__main__.D object at 0x7f9f6c18bc88>


In [22]:
d.ping()

ping: <__main__.D object at 0x7f9f6c18bc88>
post-ping: <__main__.D object at 0x7f9f6c18bc88>


* 如果要呼叫父類別可以使用 super() 內建函示
* 或是直接呼叫父類別，建議使用 super 因為遵循 MRO

## 檢查類別中 MRO 屬性

In [24]:
bool.__mro__

(bool, int, object)

In [28]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [29]:
print_mro(bool)

bool, int, object


In [31]:
from collections.abc import MutableSequence

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1> 要成為 MutableSequence 子類別必須實作 __setitem__ 方法(洗牌用)
        self._cards[position] = value

    def __delitem__(self, position):  # <2> 要成為 MutableSequence 子類別必須實作 __delitem__ 方法
        del self._cards[position]

    def insert(self, position, value):  # <3> 要成為 MutableSequence 子類別必須實作 insert 方法
        self._cards.insert(position, value)

In [32]:
print_mro(FrenchDeck2)

FrenchDeck2, MutableSequence, Sequence, Reversible, Collection, Sized, Iterable, Container, object


In [34]:
import numbers
print_mro(numbers.Integral)

Integral, Rational, Real, Complex, Number, object


In [35]:
import io
print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [36]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


* bool 從 int object 繼承方法與屬性
* FrenchDeck2 繼承來自 collection.abc 模組
* numbers.Integral 可以看到 numbers 模組提供數字 ABC
* io 模組也有 ABC 字尾加上 Base

## 總結多重繼承要點
### 介面繼承與實作繼承分開
* 介面繼承會建立一個子型態意味著父子關係
* 實作繼承時候可以避免再利用時產生程式碼重覆
### 用 ABC 來讓介面更明確
* 如果一個類別目的是定義介面，他應該是要為一個 ABC
### 使用 Mixins 來再利用程式碼
* 如果一個類別是設計來提供方法實作，來讓其他沒關係的子類別再利用，他必須是顯示 mixin 類別
* mixin 永遠不該被實例化，且具體類別不該只繼承一個 mixin
### 用名稱來讓 mixin 更明確
* 因 python 沒有正規聲明，強烈建議命名時加上 mixin -> PackMixin
### ABC 可能是 Mixin 但是返過來不一定成立
### 不要繼承高過一個具體類別
* 具體類別應該只有零個或一個其他應該都是 ABC
```
class a(b,c,d)
```
如果 b 是具體類別 cd 應該都是 ABC 或是 mixin
### 提供聚合類別給使用者
* 如果有某些 ABC 或 mixin 組合特別好用就整合一起稱之為聚合類別
### 比起類別繼承物件組合更好